# Visium pipeline runthrough
------
In this notebook, we'll go through how to utilize Giotto for visium data. While there are some notebooks that will use visium data in this tutorial, the majority use the seqFISH dataset. You can use this notebook or the script ```vis_obj.R``` as reference.

First, we'll install Giotto:

In [ ]:
remotes::install_github("RubD/Giotto@cless")

And load Giotto:

In [ ]:
library(Giotto)

Now we can define our expression, location, and python paths:

In [ ]:
# expresstion path
expr_path = system.file("extdata", "visium_DG_expr.txt", package = 'Giotto')
# spatial locations path
loc_path = system.file("extdata", "visium_DG_locs.txt", package = 'Giotto')
# python path
my_instructions = createGiottoInstructions(python_path = "/srv/conda/envs/notebook/bin/python")

And finally, we can create our Giotto object:

In [ ]:
my_visium_object <- createGiottoObject(raw_exprs = expr_path,
                                       spatial_locs = loc_path,
                                       instructions = my_instructions)

Now we can read in the image associated with this dataset:

In [ ]:
# read image
png_path = system.file("extdata", "deg_image.png", package = 'Giotto')
mg_img = magick::image_read(png_path)

First, we can look at the image and we'll see that we need to fix the alignment:

In [ ]:
mypl = spatPlot(my_visium_object, return_plot = T, point_alpha = 0.8)

In [ ]:
# test and modify image alignment
orig_png = createGiottoImage(gobject = my_visium_object, mg_object = mg_img, name = 'image',
                             xmax_adj = 450, xmin_adj = 550,
                             ymax_adj = 200, ymin_adj = 200)
mypl_image = addGiottoImageToSpatPlot(mypl, orig_png)
mypl_image


And now we can add the image to our Giotto object:

In [ ]:
# add image to Giotto object

image_list = list(orig_png)
my_visium_object = addGiottoImage(gobject = my_visium_object,
                             images = image_list)
showGiottoImageNames(my_visium_object)

We can inspect our data to explore gene and cell distribution as well as combinations:

In [ ]:
# explore gene and cell distribution
filterDistributions(my_visium_object, detection = 'genes')
filterDistributions(my_visium_object, detection = 'cells')
filterCombinations(my_visium_object,
                   expression_thresholds = c(1),
                   gene_det_in_min_cells = c(20, 20, 50, 50),
                   min_det_genes_per_cell = c(100, 200, 100, 200))

Now we filter and normalize our object:

In [ ]:
# filter and normalize
my_visium_object <- filterGiotto(gobject = my_visium_object,
                            expression_threshold = 1,
                            gene_det_in_min_cells = 50,
                            min_det_genes_per_cell = 100,
                            expression_values = c('raw'),
                            verbose = T)
my_visium_object <- normalizeGiotto(gobject = my_visium_object, scalefactor = 6000, verbose = T)

We'll add cell and gene statistics below:

In [ ]:
my_visium_object <- addStatistics(gobject = my_visium_object)

Calculate highly variable genes:

In [ ]:
my_visium_object <- calculateHVG(gobject = my_visium_object)

Run and visualize our PCA:

In [ ]:
my_visium_object <- runPCA(gobject = my_visium_object)
screePlot(my_visium_object, ncp = 30)[0]
plotPCA(gobject = my_visium_object)[0]

UMAP:

In [ ]:
my_visium_object <- runUMAP(my_visium_object, dimensions_to_use = 1:10)
plotUMAP(gobject = my_visium_object)[0]

And tSNE

In [ ]:
my_visium_object <- runtSNE(my_visium_object, dimensions_to_use = 1:10)
plotTSNE(gobject = my_visium_object)[0]

Now we can perform some clustering and inspect our data table to show which cluster each observation belongs to:

In [ ]:
# create nearest network before we run any clustering
my_visium_object <- createNearestNetwork(gobject = my_visium_object, dimensions_to_use = 1:10, k = 20)
# leiden cluster
my_visium_object <- doLeidenCluster(gobject = my_visium_object, resolution = 0.4, n_iterations = 1000)

# inspect data
head(pDataDT(my_visium_object), 10)

Below we can look at some diferent visualizations:

In [ ]:
# visualize UMAP cluster results
plotUMAP(gobject = my_visium_object, cell_color = 'leiden_clus', show_NN_network = T, point_size = 2.5)

In [ ]:
# visualize UMAP and spatial results
spatDimPlot(gobject = my_visium_object, cell_color = 'leiden_clus', spat_point_shape = 'voronoi')

In [ ]:
# visualize heatmap
showClusterHeatmap(gobject = my_visium_object, cluster_column = 'leiden_clus')

In [ ]:
# visualize dendrogram
showClusterDendrogram(my_visium_object, h = 0.5, rotate = T, cluster_column = 'leiden_clus')